1. pdf, word와 같은 문서의 내용을 읽는다.
2. 문서를 쪼갠다(split)
    - 토큰수 초과로 답변을 생성하지 못할 수도 있고
    - 문서가 길면 (input이 길면) 답변 생성이 오래 걸림
        1. chunk_size
        - 나누어진 텍스트 청크(조각)의 최대 문자 수를 지정합니다.
        - 각 청크는 이 크기를 초과하지 않습니다.

        - LLM(Large Language Model)이 처리할 수 있는 입력 크기를 제한하기 위해 사용됩니다.
            - 예를 들어, GPT-3.5의 경우 최대 토큰 수가 4096이므로 적절히 설정해야 합니다.
        2. chunk_overlap
        - 인접한 텍스트 청크 간에 중첩될 문자의 수를 지정합니다.
        - 한 청크의 끝부분 일부가 다음 청크의 시작 부분에 중복됩니다.
        
        - 청크 간의 문맥을 유지하기 위해 사용됩니다. 중첩이 없으면 문장이 잘리거나 의미가 손실될 수 있습니다.
            - 예를 들어, 문장이 한 청크의 끝에서 잘리고 다음 청크의 시작에 포함되지 않으면, 모델이 문맥을 올바르게 이해하지 못할 수 있습니다.
3. 임베딩(데이터를 숫자 형태의 벡터 형식으로 변환) -> 벡터 데이터베이스에 저장
4. 질문이 있을 때, 벡터 데이터베이스에 유사도 검색
5. 유사도 검색으로 가져온 문서를 LLM에 질문과 같이 전달 

In [ ]:
%pip install -qU langchain_community pypdf

In [ ]:
%cp /mnt/c/Users/suholee/Downloads/split/split-1.pdf /root/llm-application/


In [ ]:
from langchain_community.document_loaders import PyPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter

# 텍스트 스플릿 (몇개씩 자를 것인지)
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1500,
    chunk_overlap=200,
)

loader = PyPDFLoader(
    "./split-1.pdf",
)
document_list = loader.load_and_split(text_splitter=text_splitter)

In [ ]:
document_list

In [ ]:
from dotenv import load_dotenv
from langchain_openai import OpenAIEmbeddings

load_dotenv()

embedding = OpenAIEmbeddings(model="text-embedding-3-large")

In [ ]:
from langchain_chroma import Chroma

# database = Chroma.from_documents(documents=document_list, embedding=embedding, collection_name='chroma-tax', persist_directory="./chroma")
database = Chroma(collection_name='chroma-tax', persist_directory="./chroma", embedding_function=embedding)

In [ ]:
query = '참고한 텍스트에서 ssl은 번호가 몇번 인가요? 그리고 첫 문장을 그대로 작성해봐주세요. 영어여도 상관없어요'
# retrieved_docs = database.similarity_search(query, k = 3)


In [ ]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model='gpt-4o')


In [ ]:
%pip install -U langchain langchainhub --quiet

In [ ]:
from langchain import hub

prompt = hub.pull("rlm/rag-prompt")

In [ ]:
from langchain.chains import RetrievalQA

qa_chain = RetrievalQA.from_chain_type(
    llm,
    retriever=database.as_retriever()
)
chain_type_kwargs={"prompt": prompt}
ai_message = qa_chain({"query": query})
ai_message